In [1]:
import time
import numpy as np
import pinocchio as pin
from robot_descriptions.loaders.pinocchio import load_robot_description
from utils.meshcat_viewer_wrapper import MeshcatVisualizer


In [2]:
robot = load_robot_description('ur5_description')
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
q = pin.neutral(robot.model)
viz.display(q)

Given the rigid body dynamics equation we learned from the lecture:
$$
M(q)\ddot{q} = \tau(t) - h(q, \dot{q}) - J_{ee}^{\mathrm{T}}(q)\mathcal{F}_{ee}
$$
We can use pinocchio to calculate the mass matrix M via the composite rigid body algorithm and the nonlinear effects h.

In [4]:
vq = np.zeros(robot.model.nv)
M = pin.crba(robot.model, robot.data, q)
h = pin.nle(robot.model, robot.data, q, vq)

In order to simulate the forward dynamics we can solve directly for $\ddot{q}$:
$$
\ddot{q} = M(q)^{-1}(\tau(t) - h(q, \dot{q}) - J_{ee}^{\mathrm{T}}(q)\mathcal{F}_{ee})
$$
This can be done directly with pinoccio by using the articulated body algorithm aba. Read through the loop below and run the cell to see the dynamics simulation.

In [5]:
q = np.array([0,-np.pi/2,0,0,0,0])
vq = np.zeros(robot.model.nv)
viz.display(q)

dt = 2e-3
N_steps = 5000

for it in range(N_steps):
    t = it*dt

    # Compute the force that apply
    tauq = np.zeros(robot.model.nv)

    # Use aba forward dynamics to calculate aq
    aq = pin.aba(robot.model, robot.data, q, vq, tauq)

    # Double integration to update vq and q
    vq += aq * dt
    q = pin.integrate(robot.model, q, vq * dt)

    # Visualization
    if it%20==0: 
        viz.display(q)
        time.sleep(20*dt)

Now lets add friction to the dynamics. Fill in the missing code for aq. This time don't use the aba, but write it directly so you can add a friction term $\mathcal{K}\dot{q}$ to simulate the dynamics with friction. 

In [6]:
q = np.array([0,-np.pi/2,0,0,0,0])
vq = np.zeros(robot.model.nv)
viz.display(q)

dt = 2e-3
N_steps = 5000
Kf = 0.3

for it in range(N_steps):
    t = it*dt

    M = pin.crba(robot.model, robot.data, q)
    b = pin.nle(robot.model, robot.data, q, vq)

    # Zero torque applied at the joints
    tauq = np.zeros(robot.model.nv)

    # Dynamics equation with friction
    aq = np.linalg.pinv(M) @ (tauq - Kf * vq - b)
    # Double integration to update vq and q
    vq += aq * dt
    q = pin.integrate(robot.model, q, vq * dt)

    # Visualization
    if it%20==0: 
        viz.display(q)
        time.sleep(20*dt)

We can also solve the inverse dynamics using the recursive newton euler algorithm. This will find the torques $\tau$ for a given $(q, \dot{q}, \ddot{q}, \mathcal{F_{ee}})$ in our case $\mathcal{F_{ee}} = \mathbf{0}$.
$$
\tau = M(q)\ddot{q} + h(q, \dot{q})
$$

Run the cell below to see the dynamics simulation with inverse dynamics to hold the initial configuration.

In [7]:
q = np.array([0,-np.pi/2,np.pi/2,0,0,0])
vq = np.zeros(robot.model.nv)
# Set the desired joint values
q_des = q
vq_des = np.zeros(robot.model.nv)
aq_des = vq = np.zeros(robot.model.nv)
viz.display(q)

dt = 2e-3
N_steps = 5000
Kf = 0.3

for it in range(N_steps):
    t = it*dt

    # Compute the torque using inverse dynamics
    tauq = pin.rnea(robot.model, robot.data, q_des, vq_des, aq_des)

    aq = pin.aba(robot.model, robot.data, q, vq, tauq)
    # Double integration to update vq and q
    vq += aq * dt
    q = pin.integrate(robot.model, q, vq * dt)

    # Visualization
    if it%20==0: 
        viz.display(q)
        time.sleep(20*dt)

The robot eventually diverges when computing torques this way. What are the problems with this approach? What could be done differently?